<a href="https://colab.research.google.com/github/Vinayak-Sharma12/Types-of-Vector-Stores-or-Databases/blob/main/PineCone_VectorDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU langchain langchain-pinecone  langchain_community

In [2]:
from google.colab import userdata
api_key=userdata.get('pinecone_api_key')

## **CREATE PINECONE CLIENT**

In [3]:
from pinecone import Pinecone
pc=Pinecone(api_key=api_key)

## **CREATING SERVERLESS INDEX**

In [4]:
from pinecone import ServerlessSpec
index_name='my-index'
if not pc.has_index(index_name):
    pc.create_index(
         name=index_name,
         dimension=384, #Whatever the dimension is of your embedding
         metric='cosine',
         spec=ServerlessSpec(cloud="aws",region='us-east-1'),
          )
index=pc.Index(index_name)

## **DEFINE EMBEDDING MODEL**

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings

# Wrap SentenceTransformer in LangChain embeddings
embeddings= HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/tmp/ipython-input-2084042806.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings= HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or d

## **CREATE VECTOR STORE**

In [6]:
from langchain_pinecone import PineconeVectorStore
vector_db=PineconeVectorStore(index=index,embedding=embeddings)

ADD CHUNKS TO VECTOR DB

In [12]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

In [15]:
vector_db.add_documents(documents=documents)

['5069af67-f1a1-4ddd-aa6e-5f155c2baafb',
 'ffb30e48-ff99-4820-9b05-fce17ec10d30',
 '6622f745-52fa-486f-ad99-d2dbe0757316',
 'c202545d-fe60-4c2a-8cce-e5c9cb41b32a',
 '9bb39158-7a42-4ae5-9d8e-0a855c52382c',
 '2a16359d-7565-42a9-9048-cafe2854072f',
 'd53529b3-bcab-474b-afad-337cd9ff2b06',
 '60ec50f7-e2d9-4316-b49e-f3e6c66e6635',
 '7c4a5ea5-fc86-4298-9b92-e849adb7a2bf',
 '29cc0e6f-b185-4a1b-9f0e-06bc883eeabb']

## **QUERY THE DATABASE**

In [17]:
results=vector_db.similarity_search_with_score(
    query="Who is the Iphone price",
    k=2
)
print(results)

[(Document(id='761488a8-210e-445b-bc28-17fd6844694e', metadata={'source': 'website'}, page_content='Is the new iPhone worth the price? Read this review to find out.'), 0.608708262), (Document(id='9feca119-bbed-422e-9fb7-2c547d43d106', metadata={'source': 'website'}, page_content='Is the new iPhone worth the price? Read this review to find out.'), 0.608708262)]
